# Mech Tools Evaluation

### Create the dataset

In [32]:
# !docker build -t etl:0.0.1 . && docker run etl:0.0.1

### Load the dataset

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option("display.precision", 2)
pd.set_option('display.max_columns', None)

In [2]:
dataset = pd.read_csv("dataset.csv")
# drop duplicates
dataset = dataset.drop_duplicates(subset='request_id')

# choose only non error
dataset = dataset[dataset['error'] == False].reset_index(drop=True)

str_cols = ("id", "currentAnswer", "title", "request_id", "prompt_request", "tool", "nonce", "vote")
for col in str_cols:
    dataset[col] = dataset[col].astype("string")
dataset.head()



,id,currentAnswer,title,request_id,request_block,prompt_request,tool,nonce,deliver_block,error,error_message,prompt_response,p_yes,p_no,confidence,info_utility,vote,win_probability
0,0x0094fa304017d5c2b355790e2976f769ea600492,No,Will the Hisense U8K be considered a top-tier ...,5545075191266518558547029150600135891156080971...,29552624,"Given the information available, and consideri...",prediction-online,4150b2a6-a1a4-43cd-b346-8dc5c5173d0b,29560402,False,NaN,NaN,0.7,0.3,0.8,0.6,Yes,0.7
1,0x0094fa304017d5c2b355790e2976f769ea600492,No,Will the Hisense U8K be considered a top-tier ...,4420322125272725217473329489276855755733870871...,29552737,"With the given question ""Will the Hisense U8K ...",prediction-online,4e81f577-6063-45bb-a26e-2c939c111cf3,29560222,False,NaN,NaN,0.6,0.4,0.8,0.5,Yes,0.6
2,0x0094fa304017d5c2b355790e2976f769ea600492,No,Will the Hisense U8K be considered a top-tier ...,8092041810453425517708884760497898797449114282...,29553033,"With the given question ""Will the Hisense U8K ...",prediction-online,ceba7701-4a3d-4720-bcaf-659666981d51,29560062,False,NaN,NaN,0.6,0.4,0.8,0.9,Yes,0.6
3,0x0094fa304017d5c2b355790e2976f769ea600492,No,Will the Hisense U8K be considered a top-tier ...,1109136049191103125504131999953988293599112824...,29553373,"Given the information available, and consideri...",prediction-online,2d8d2776-2a71-4e6d-a450-a85e0e1ec64a,29559888,False,NaN,NaN,0.6,0.4,0.8,0.4,Yes,0.6
4,0x0094fa304017d5c2b355790e2976f769ea600492,No,Will the Hisense U8K be considered a top-tier ...,5385201172829803617221747434686259018721904083...,29553477,"With the given question ""Will the Hisense U8K ...",prediction-online,641be53c-5304-44b1-ad6f-595d3b0ce319,29559855,False,NaN,NaN,0.6,0.4,0.8,0.5,Yes,0.6


In [3]:
dataset.shape

(68733, 18)

In [4]:
dataset.describe()

,request_block,deliver_block,p_yes,p_no,confidence,info_utility,win_probability
count,6.87e+04,6.87e+04,68733.00,68733.00,68733.00,68733.00,68733.00
mean,3.07e+07,3.07e+07,0.48,0.52,0.78,0.56,0.67
std,6.86e+05,6.86e+05,0.20,0.20,0.08,0.19,0.10
min,2.89e+07,2.89e+07,0.00,0.00,0.00,0.00,0.50
25%,3.01e+07,3.01e+07,0.30,0.40,0.80,0.50,0.60
50%,3.06e+07,3.06e+07,0.60,0.40,0.80,0.60,0.70
75%,3.13e+07,3.13e+07,0.60,0.70,0.80,0.60,0.75
max,3.20e+07,3.20e+07,1.00,1.00,1.00,1.00,1.00


In [5]:
dataset["vote"].describe()

count     63653
unique        2
top         Yes
freq      34992
Name: vote, dtype: object

### Normalize confidences

In [6]:
dataset["confidence"].unique()

array([0.8 , 0.9 , 0.7 , 0.5 , 0.1 , 0.6 , 0.75, 0.85, 0.3 , 0.4 , 1.  ,
       0.2 , 0.35, 0.  , 0.65, 0.99, 0.95, 0.55, 0.62, 0.88, 0.64, 0.81,
       0.68])

In [7]:
# number of samples with confidence below 0.5
print(f"Number of samples with confidence below 0.5: {dataset[dataset['confidence'] < 0.5].shape[0]}")

# drop samples with confidence below 0.5
dataset = dataset[dataset['confidence'] >= 0.5].reset_index(drop=True)

Number of samples with confidence below 0.5: 44


In [8]:
# bucket confidence into 0.5, 0.6, 0.7, 0.8, 0.9
dataset['confidence'] = dataset['confidence'].apply(lambda x: round(x, 1))

# if confidence is 1 --> 0.9
dataset['confidence'] = dataset['confidence'].apply(lambda x: 0.9 if x == 1.0 else x)
dataset["confidence"].unique()

array([0.8, 0.9, 0.7, 0.5, 0.6])

### Check the percentage of wins vs confidence for all the tools

In [9]:
def accuracy(data):
    correct_answers_mask = data["currentAnswer"] == data["vote"]
    n_answers = correct_answers_mask.count()
    n_answers_success = correct_answers_mask.sum()

    if n_answers == 0:
        accuracy = None
    elif n_answers_success == 0:
        accuracy = 0
    else:
        accuracy = n_answers_success/n_answers * 100

    return pd.Series({"n_correct": n_answers_success, "n_pred": n_answers, "accuracy": accuracy})

In [10]:
def acc_per_tool(group, col: str, conf: float):
    return group.apply(lambda x: accuracy(x[x[col] == conf]))

In [11]:
def gen_stats(group, col: str):
    stats = {f"{col}_{prob}" : acc_per_tool(group, col, prob) for prob in sorted(dataset[col].unique())}
    stats["total"] = tools_group.apply(accuracy)
    return pd.concat(stats.values(), axis=1, keys=stats.keys())

In [12]:
tools_group = dataset.groupby("tool")
tools_stats_per_conf = gen_stats(tools_group, "confidence")
display(tools_stats_per_conf)

confidence_0.5                   \
                                                n_correct  n_pred accuracy   
tool                                                                         
claude-prediction-offline                           626.0  1068.0    58.61   
claude-prediction-online                            194.0   407.0    47.67   
prediction-offline                                    0.0     0.0      NaN   
prediction-offline-sme                                0.0     0.0      NaN   
prediction-online                                     0.0     0.0      NaN   
prediction-online-sme                                 1.0     3.0    33.33   
prediction-online-summarized-info                     0.0     0.0      NaN   
prediction-sentence-embedding-bold                    0.0     0.0      NaN   
prediction-sentence-embedding-conservative            0.0     0.0      NaN   

                                           confidence_0.6                  \
                                                n_correct n_pred accuracy   
tool                                                                        
claude-prediction-offline                            52.0   94.0    55.32   
claude-prediction-online                            233.0  430.0    54.19   
prediction-offline                                   17.0   30.0    56.67   
prediction-offline-sme                              108.0  182.0    59.34   
prediction-online                                   136.0  194.0    70.10   
prediction-online-sme                               310.0  653.0    47.47   
prediction-online-summarized-info                     1.0    2.0    50.00   
prediction-sentence-embedding-bold                    0.0    0.0      NaN   
prediction-sentence-embedding-conservative            0.0    0.0      NaN   

                                           confidence_0.7                   \
                                                n_correct  n_pred accuracy   
tool                                                                         
claude-prediction-offline                           172.0   396.0    43.43   
claude-prediction-online                            440.0   778.0    56.56   
prediction-offline                                  161.0   298.0    54.03   
prediction-offline-sme                              357.0   516.0    69.19   
prediction-online                                  2076.0  3134.0    66.24   
prediction-online-sme                              2593.0  5381.0    48.19   
prediction-online-summarized-info                     7.0    13.0    53.85   
prediction-sentence-embedding-bold                   22.0    35.0    62.86   
prediction-sentence-embedding-conservative            0.0     0.0      NaN   

                                           confidence_0.8                    \
                                                n_correct   n_pred accuracy   
tool                                                                          
claude-prediction-offline                            58.0    106.0    54.72   
claude-prediction-online                             72.0    175.0    41.14   
prediction-offline                                  689.0   1590.0    43.33   
prediction-offline-sme                              890.0   1875.0    47.47   
prediction-online                                 10287.0  22415.0    45.89   
prediction-online-sme                              9325.0  18158.0    51.35   
prediction-online-summarized-info                    47.0    102.0    46.08   
prediction-sentence-embedding-bold                   16.0     17.0    94.12   
prediction-sentence-embedding-conservative           18.0     35.0    51.43   

                                           confidence_0.9                   \
                                                n_correct  n_pred accuracy   
tool                                                                         
claude-prediction-offline                            78.0    

### Check the percentage of wins vs probability for all the tools

In [13]:
tools_stats_per_prob = gen_stats(tools_group, "win_probability")
tools_stats_per_prob

win_probability_0.5         \
                                                     n_correct n_pred   
tool                                                                    
claude-prediction-offline                                  0.0    0.0   
claude-prediction-online                                   0.0    0.0   
prediction-offline                                         0.0    0.0   
prediction-offline-sme                                     0.0    0.0   
prediction-online                                          0.0    0.0   
prediction-online-sme                                      0.0    0.0   
prediction-online-summarized-info                          0.0    0.0   
prediction-sentence-embedding-bold                         0.0    0.0   
prediction-sentence-embedding-conservative                 0.0    0.0   

                                                    win_probability_0.505  \
                                           accuracy             n_correct   
tool                                                                        
claude-prediction-offline                       NaN                   0.0   
claude-prediction-online                        NaN                   0.0   
prediction-offline                              NaN                   0.0   
prediction-offline-sme                          NaN                   0.0   
prediction-online                               NaN                   0.0   
prediction-online-sme                           NaN                   0.0   
prediction-online-summarized-info               NaN                   0.0   
prediction-sentence-embedding-bold              NaN                   0.0   
prediction-sentence-embedding-conservative      NaN                   0.0   

                                                            \
                                           n_pred accuracy   
tool                                                         
claude-prediction-offline                     0.0      NaN   
claude-prediction-online                      1.0      0.0   
prediction-offline                            0.0      NaN   
prediction-offline-sme                        0.0      NaN   
prediction-online                             0.0      NaN   
prediction-online-sme                         0.0      NaN   
prediction-online-summarized-info             0.0      NaN   
prediction-sentence-embedding-bold            0.0      NaN   
prediction-sentence-embedding-conservative    0.0      NaN   

                                           win_probability_0.525         \
                                                       n_correct n_pred   
tool                                                                      
claude-prediction-offline                                    0.0    0.0   
claude-prediction-online                                     0.0    0.0   
prediction-offline                                           0.0    0.0   
prediction-offline-sme                                       0.0    0.0   
prediction-online                                            0.0    0.0   
prediction-online-sme                                        0.0    1.0   
prediction-online-summarized-info                            0.0    0.0   
prediction-sentence-embedding-bold                           0.0    0.0   
prediction-sentence-embedding-conservative                   0.0    0.0   

                                                    win_probability_0.53  \
                                           accuracy            n_correct   
tool                                                                       
claude-prediction-offline                       NaN                  0.0   
claude-prediction-online                        NaN                  2.0   
prediction-offline                              NaN                  0.0   
prediction-offline-sme                          NaN                  0.0   
prediction-online                               NaN            

### List the well performing tools

In [14]:
# we are using 55% as the performance threshold and 100 as the samples threshold
perf_threshold = 55
samples_threshold = 100

In [15]:
well_performing = tools_stats_per_conf.loc[:, ("total", "accuracy")] > perf_threshold
enough_samples = tools_stats_per_conf.loc[:, ("total", "n_pred")] > samples_threshold
tools_stats_per_conf.loc[well_performing & enough_samples, [("total", "accuracy"), ("total", "n_pred")]]

total        
                          accuracy  n_pred
tool                                      
claude-prediction-offline    56.02  1760.0